# Imports

In [2]:
import importlib
import classification_training_utils
importlib.reload(classification_training_utils)
import utils
importlib.reload(utils)

import os
import pandas as pd
import json
import torch
import sys
import pickle
from datetime import datetime

sys.path.append('../S3BERT/src')
from training_utils import create_weighted_avg, normalize_column, get_data, train_combined

0
Device:  cpu


In [3]:
import custom_losses
import custom_evaluators
from sentence_transformers import SentenceTransformer, InputExample
import model_freeze as freeze
import prediction_helpers as ph
import ast
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoModel
from training_utils import get_data, create_weighted_avg, normalize_column, train, infer, get_mse_scores, get_spearman_scores
from training_utils import prepare_for_training as similarity_prepare_for_training
from classification_training_utils import get_big_consulting_df, collect_classification_labels, get_relevant_classifications, train, filter_relevant_classifications, get_top_values, get_news_df
from classification_training_utils import get_train_objectives as get_classification_train_objectives
from sentence_transformers import losses
from sentence_transformers import evaluation
from utils import load_model

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

print(torch.cuda.device_count())
print("Device: ", device)

0
Device:  cpu


# Training

In [4]:
# params = {}
# params["EPOCHS"] = 5
# params["SAVE_PATH"] = "outputs/" #"s3bert_" + params["SBERT_INIT"] + "/
# params["BATCH_SIZE"] = 32
# params["WARMUP_STEPS"] = 10
# params["EVAL_STEPS"] = 100
# params["LEARNING_RATE"] = 2e-5
# params["DATASETS"] = ["consulting"]

# # similarity training specific
# params["USE_WEIGHTED_SIMILARITY"] = True
# params["USE_MANUAL_LABELS"] = True # whether to use val-test labels that I created manually or some similarity metric
# params["LOSSES"] = ["cosine"] # ["consistency", "distill", "distill2", "cosine"]
# params["FEATURES"] = ['input1', 'input2']
# params["USE_DISTILL"] = any('distill' in loss for loss in params["LOSSES"])
# if params["USE_DISTILL"]:
#     params["FEATURES"] += ['company', 'country', 'classification', 'keywords']
# # FEATURES += ['similarity']
# params["N"] = len(params["FEATURES"]) - 2 # number of metrics
# params["FEATURE_DIM"] = 16 if params["USE_DISTILL"] else 0

# # classification training specific
# params["USE_REPLACE_DATA"] = False
# params["USE_ORIGINAL_DATA"] = True
# params["UNFREEZE_LAYERS"] = 2
# params["EXCLUDE_ENTITY_OTHER"] = True
# params["INITIALIZED_MODEL"] = None #"brjezierski/S3BERT" # "intfloat/e5-small-v2"
# params["OCCURENCE_CUTOFF"] = 2
# params["CREATE_NEW_SPLIT"] = False

# dataset_dir = "../classification-training-data/"
# model = load_model() if not params["INITIALIZED_MODEL"] else load_model(
#         model=params["INITIALIZED_MODEL"])

In [5]:
# column_list = ['country', 'company', 'keywords', 'classification']
# column_proportions = (10, 2, 1, 1, 1)
# similarity_train_df, similarity_val_df, similarity_test_df = get_data(column_list, column_proportions, params, current_prefix='../similarity-training-data/sbert-company/')
# similarity_train_df[:1]

In [6]:
# big_consulting_df = get_big_consulting_df(params)
# big_consulting_df = collect_classification_labels(big_consulting_df, verbose=True)
# big_consulting_df = get_top_values(big_consulting_df, params)
# all_classifications, relevant_classifications = get_relevant_classifications(big_consulting_df, params)
# big_consulting_df, classifications = filter_relevant_classifications(big_consulting_df, all_classifications, relevant_classifications)

In [7]:
# similarity_train_objectives, similarity_evaluator = similarity_prepare_for_training(model, params, similarity_train_df, similarity_val_df)

# classification_train_objectives, dev_classification_evaluator, test_similarity_evaluator = classification_prepare_for_training(big_consulting_df, classifications, params, dataset_dir, model)

# train_combined(classification_train_objectives, similarity_train_objectives, similarity_evaluator, dev_classification_evaluator, params, model)

In [8]:
def run_pipeline(column_list, column_proportions, save_model=False):
    model = load_model() if not params["INITIALIZED_MODEL"] else load_model(
        model=params["INITIALIZED_MODEL"])
    
    # prepare for training - classification
    params["DATASETS"] = params["CLASSIFICATION_DATASETS"]
    if "consulting" in params["DATASETS"]:
        big_consulting_df = get_big_consulting_df(params)
        big_consulting_df = collect_classification_labels(big_consulting_df)
        big_consulting_df = get_top_values(big_consulting_df, params)
        big_consulting_all_classifications, big_consulting_relevant_classifications = get_relevant_classifications(big_consulting_df, params)
        big_consulting_df, big_consulting_classifications = filter_relevant_classifications(big_consulting_df, big_consulting_all_classifications, big_consulting_relevant_classifications)
        params["WARMUP_STEPS"] = int(len(big_consulting_df) * params["EPOCHS"] * 0.1)

    if "ai" in params["DATASETS"]:
        ai_news_df = get_news_df(params, 'ai_news')
        ai_news_df = collect_classification_labels(ai_news_df)
        ai_news_df = get_top_values(ai_news_df, params)
        ai_news_all_classifications, ai_news_relevant_classifications = get_relevant_classifications(ai_news_df, params)
        ai_news_df, ai_news_classifications = filter_relevant_classifications(ai_news_df, ai_news_all_classifications, ai_news_relevant_classifications)
        params["WARMUP_STEPS"] = int(len(ai_news_df) * params["EPOCHS"] * 0.1)

    if "car" in params["DATASETS"]:
        car_news_df = get_news_df(params, 'car_news')
        # How about duplicating car news dataset?
        car_news_df = collect_classification_labels(car_news_df)
        car_news_df = get_top_values(car_news_df, params)
        car_news_all_classifications, car_news_relevant_classifications = get_relevant_classifications(car_news_df, params)
        car_news_df, car_news_classifications = filter_relevant_classifications(car_news_df, car_news_all_classifications, car_news_relevant_classifications)
        params["WARMUP_STEPS"] = int(len(car_news_df) * params["EPOCHS"] * 0.1)

    embeddings_prefix = '../glanos-data/embeddings/'

    if params['SNIPPET_COLUMN_NAME'] == "replace_no_tags":

        if "ai" in params["DATASETS"]:
            with open(f'{embeddings_prefix}ai_news_replace_no_tags.pickle', 'rb') as f:
                ai_news_no_tags_embeddings = pickle.load(f)
            ai_news_df = create_replace_no_tags_embeddings(ai_news_df, ai_news_no_tags_embeddings)
            ai_news_df['embedding'] = ai_news_df['replace_no_tags'].map(ai_news_no_tags_embeddings)

        if "car" in params["DATASETS"]:
            with open(f'{embeddings_prefix}car_news_replace_no_tags.pickle', 'rb') as f:
                car_news_no_tags_embeddings = pickle.load(f)
            car_news_df = create_replace_no_tags_embeddings(car_news_df, car_news_no_tags_embeddings)
            car_news_df['embedding'] = car_news_df['replace_no_tags'].map(car_news_no_tags_embeddings)

        if "consulting" in params["DATASETS"]:
            with open(f'{embeddings_prefix}big_consulting_2_replace_no_tags.pickle', 'rb') as f:
                consulting_no_tags_embeddings = pickle.load(f)
            big_consulting_df = create_replace_no_tags_embeddings(big_consulting_df, consulting_no_tags_embeddings)
            big_consulting_df['embedding'] = big_consulting_df['replace_no_tags'].map(consulting_no_tags_embeddings)

    classifications_path =  "../classification-training-data/classifications.pkl"

    with open(classifications_path, 'rb') as file:
        classifications = pickle.load(file)

    training_datasets = []
    classification_lists = []
    if "consulting" in params["DATASETS"]:
        training_datasets.append(big_consulting_df)
        classification_lists.append(big_consulting_classifications)
    if "ai" in params["DATASETS"]:
        ai_news_df.drop(columns=['embedding'])
        if params["DATASETS"].count("ai") == 2:
            ai_news_df = pd.concat([ai_news_df, ai_news_df], axis=0).reset_index(drop=True) 
        training_datasets.append(ai_news_df)
        classification_lists.append(ai_news_classifications)
    if "car" in params["DATASETS"]:
        car_news_df.drop(columns=['embedding'])
        if params["DATASETS"].count("car") == 2:
            car_news_df = pd.concat([car_news_df, car_news_df], axis=0).reset_index(drop=True) 
        training_datasets.append(car_news_df)
        classification_lists.append(car_news_classifications)

    print("For clustering using datasets:", params["DATASETS"])

    classification_train_objectives, classification_dev_evaluator, classification_test_evaluator, steps_per_epoch, model = get_classification_train_objectives(training_datasets, classification_lists, params, dataset_dir, model, save_model=True)
    output_path = ("classification_train_output/sbert-" +
                   datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

    # prepare for training - similarity
    params["DATASETS"] = params["SIMILARITY_DATASETS"]
    print("For similarity using datasets:", params["DATASETS"])
    similarity_train_df, similarity_val_df, similarity_test_df = get_data(column_list, column_proportions, params, current_prefix=params["SIMILARITY_DATASET_DIR"])
    similarity_train_objectives, similarity_evaluator = similarity_prepare_for_training(model, params, similarity_train_df, similarity_val_df)

    steps_per_epoch = len(similarity_train_df)/params["BATCH_SIZE"]
    params["EVAL_STEPS"] = 100 if steps_per_epoch <= 200 else steps_per_epoch/4
    train_combined(classification_train_objectives, similarity_train_objectives, similarity_evaluator, classification_dev_evaluator, params, model)

    print('Classification score', model.evaluate(classification_dev_evaluator))

    frozen_model = load_model() if not params["INITIALIZED_MODEL"] else load_model(
        model=params["INITIALIZED_MODEL"])
    print('Classification baseline', frozen_model.evaluate(classification_dev_evaluator))
    
    eval_model_path = "./" + params["SAVE_PATH"] + "/" # local
    test_df_wpred = infer(similarity_test_df, eval_model_path, params)
    print('Spearman')
    get_spearman_scores(test_df_wpred, params)
    print('MSE')
    get_mse_scores(test_df_wpred, params)

    print('Setting label to weighted_similarity')
    print('MSE')
    test_df2 = similarity_test_df.copy()
    test_df2['label'] = similarity_test_df['weighted_similarity']
    test_df_wpred = infer(test_df2, eval_model_path, params)
    print('Spearman')
    get_spearman_scores(test_df_wpred, params)
    print('MSE')
    get_mse_scores(test_df_wpred, params)


# Experiment 1

In [49]:
# - combined consulting sim and ai+car class
params = {}
params["EPOCHS"] = 1
params["SAVE_PATH"] = "outputs/" #"s3bert_" + params["SBERT_INIT"] + "/
params["BATCH_SIZE"] = 32
params["WARMUP_STEPS"] = 10
params["EVAL_STEPS"] = 100
params["LEARNING_RATE"] = 2e-5
params["SIMILARITY_DATASETS"] = ["consulting"]

# similarity training specific
params["USE_WEIGHTED_SIMILARITY"] = True
params["USE_MANUAL_LABELS"] = True # whether to use val-test labels that I created manually or some similarity metric
params["LOSSES"] = ["cosine"] # ["consistency", "distill", "distill2", "cosine"]
params["FEATURES"] = ['input1', 'input2']
params["USE_DISTILL"] = any('distill' in loss for loss in params["LOSSES"])
if params["USE_DISTILL"]:
    params["FEATURES"] += ['company', 'country', 'classification', 'keywords']
# FEATURES += ['similarity']
params["N"] = len(params["FEATURES"]) - 2 # number of metrics
params["FEATURE_DIM"] = 16 if params["USE_DISTILL"] else 0

# classification training specific
params["VAL_DEV_SIZE"] = 100
params["CREATE_NEW_SPLIT"] = False
params["CLASSIFICATION_DATASETS"] = ["ai", "car"]
params["SNIPPET_COLUMN_NAME"] = 'snippet'
params["UNFREEZE_LAYERS"] = 5
params["EXCLUDE_ENTITY_OTHER"] = True
params["OCCURENCE_CUTOFF"] = 3
params["INITIALIZED_MODEL"] = "sentence-transformers/all-MiniLM-L12-v2"

params["SIMILARITY_DATASET_DIR"] = '../similarity-training-data/consulting/sbert-company/'

dataset_dir = "../classification-training-data/snippet_seperate_classifications/"
model = load_model() if not params["INITIALIZED_MODEL"] else load_model(
        model=params["INITIALIZED_MODEL"])

In [50]:
column_list = ['country', 'company', 'keywords', 'classification']
column_proportions = (10, 2, 1, 1, 1)
run_pipeline(save_model=True)

# Classification score 0.85
# Classification baseline 0.82
# Loading model from ./outputs//
# Spearman
# Baseline similarity 0.9184827961965057
# Global similarity 0.9071650097986531
# MSE
# Baseline similarity 0.019708063019580228
# Global similarity 0.028256415309782915


For clustering using datasets: ['ai', 'car']
Training with multiple objectives
Reading existing dev and test triplets
e=1, embeddings from snippet, unfreezing 5 last layers, excluding Entity and Other labels, starting with sentence-transformers/all-MiniLM-L12-v2, only including words that occur at least 3 times, training data size67319

Training data size 67319
Validation data size 100
Test data size 100
Reading existing dev and test triplets
e=1, embeddings from snippet, unfreezing 5 last layers, excluding Entity and Other labels, starting with sentence-transformers/all-MiniLM-L12-v2, only including words that occur at least 3 times, training data size38635

Training data size 38635
Validation data size 100
Test data size 100
Performance before fine-tuning:
For similarity using datasets: ['consulting']
['similarity']
['similarity']


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/202 [00:00<?, ?it/s]

Score at epoch 0, step 100: 1.7519105068910281
Score at epoch 0, step 200: 1.7679788866577248
Score at epoch 0, step -1: 1.7669445681040403
Classification score 0.85
Classification baseline 0.82
Loading model from ./outputs//
Spearman
Baseline similarity 0.9184827961965057
Global similarity 0.9071650097986531
MSE
Baseline similarity 0.019708063019580228
Global similarity 0.028256415309782915


In [ ]:
# 0.85

# Experiment 2

In [56]:
# - combined-ai_car
params = {}
params["EPOCHS"] = 2
params["SAVE_PATH"] = "combined-ai_car/" #"s3bert_" + params["SBERT_INIT"] + "/
params["BATCH_SIZE"] = 32
params["WARMUP_STEPS"] = 10
params["EVAL_STEPS"] = 100
params["LEARNING_RATE"] = 2e-5
params["SIMILARITY_DATASETS"] = ["ai", "car"]

# similarity training specific
params["USE_WEIGHTED_SIMILARITY"] = True
params["USE_MANUAL_LABELS"] = False # whether to use val-test labels that I created manually or some similarity metric
params["LOSSES"] = ["cosine"] # ["consistency", "distill", "distill2", "cosine"]
params["FEATURES"] = ['input1', 'input2']
params["USE_DISTILL"] = any('distill' in loss for loss in params["LOSSES"])
if params["USE_DISTILL"]:
    params["FEATURES"] += ['company', 'classification', 'keyword']
# FEATURES += ['similarity']
params["N"] = len(params["FEATURES"]) - 2 # number of metrics
params["FEATURE_DIM"] = 16 if params["USE_DISTILL"] else 0

# classification training specific
params["VAL_DEV_SIZE"] = 100
params["CREATE_NEW_SPLIT"] = False
params["CLASSIFICATION_DATASETS"] = ["ai", "car"]
params["SNIPPET_COLUMN_NAME"] = 'snippet'
params["UNFREEZE_LAYERS"] = 5
params["EXCLUDE_ENTITY_OTHER"] = True
params["OCCURENCE_CUTOFF"] = 3
params["INITIALIZED_MODEL"] = "sentence-transformers/all-MiniLM-L12-v2"

params["SIMILARITY_DATASET_DIR"] = '../similarity-training-data/ai_car/snippet/sbert-company-0.0/'

dataset_dir = "../classification-training-data/snippet_seperate_classifications/"
model = load_model() if not params["INITIALIZED_MODEL"] else load_model(
        model=params["INITIALIZED_MODEL"])

In [57]:
column_list = ['company', 'keyword', 'classification']
column_proportions = (10, 2, 1, 1)
run_pipeline(column_list, column_proportions, save_model=True)


# epoch 1
# Classification score 0.91
# Classification baseline 0.82
# Loading model from ./combined-ai_car//
# Spearman
# Baseline similarity 1.0
# Global similarity 0.9709637740943525
# MSE
# Baseline similarity 0.0
# Global similarity 0.002940405750597529
# Setting label to weighted_similarity
# MSE
# Loading model from ./combined-ai_car//
# Spearman
# Baseline similarity 0.8550888772219306
# Global similarity 0.8422080552013802
# MSE
# Baseline similarity 0.01144891517221311
# Global similarity 0.005623157691379908



For clustering using datasets: ['ai', 'car']
Training with multiple objectives
Reading existing dev and test triplets
e=2, embeddings from snippet, unfreezing 5 last layers, excluding Entity and Other labels, starting with sentence-transformers/all-MiniLM-L12-v2, only including words that occur at least 3 times, training data size67319

Training data size 67319
Validation data size 100
Test data size 100
Reading existing dev and test triplets
e=2, embeddings from snippet, unfreezing 5 last layers, excluding Entity and Other labels, starting with sentence-transformers/all-MiniLM-L12-v2, only including words that occur at least 3 times, training data size38635

Training data size 38635
Validation data size 100
Test data size 100
Performance before fine-tuning:
For similarity using datasets: ['ai', 'car']


/Users/bartekjezierski/Desktop/master-thesis/notebooks/training_utils.py:48: DtypeWarning: Columns (14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f'{current_prefix}train.tsv', sep='\t', index_col=0)


['similarity']
['similarity']


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1207 [00:00<?, ?it/s]

Score at epoch 0, step 100: 1.6561449036225906
Score at epoch 0, step 200: 1.6624308107702692
Score at epoch 0, step 300: 1.6735010875271883
Score at epoch 0, step 400: 1.638596714917873
Score at epoch 0, step 500: 1.5636240906022651
Score at epoch 0, step 600: 1.410756768919223
Score at epoch 0, step 700: 1.3349148728718219
Score at epoch 0, step 800: 1.2770711767794194
Score at epoch 0, step 900: 1.2315292882322058
Score at epoch 0, step 1000: 1.2282194554863872
Score at epoch 0, step 1100: 1.2230375759393985
Score at epoch 0, step 1200: 1.2252043801095027
Score at epoch 0, step -1: 1.2104522613065327


Iteration:   0%|          | 0/1207 [00:00<?, ?it/s]

Score at epoch 1, step 100: 1.1956678916972925
Score at epoch 1, step 200: 1.1605250131253282
Score at epoch 1, step 300: 1.1691907297682442
Score at epoch 1, step 400: 1.196147153678842
Score at epoch 1, step 500: 1.1548661216530414
Score at epoch 1, step 600: 1.1551698792469813
Score at epoch 1, step 700: 1.1876951923798096
Score at epoch 1, step 800: 1.1773359333983349
Score at epoch 1, step 900: 1.1774656866421662
Score at epoch 1, step 1000: 1.1715060376509414
Score at epoch 1, step 1100: 1.1620100502512565
Score at epoch 1, step 1200: 1.1396602415060377
Score at epoch 1, step -1: 1.1379186979674492
Classification score 0.91
Classification baseline 0.82
Loading model from ./combined-ai_car//
Spearman
Baseline similarity 1.0
Global similarity 0.964440111002775
MSE
Baseline similarity 0.0
Global similarity 0.003135770754646344
Setting label to weighted_similarity
MSE
Loading model from ./combined-ai_car//
Spearman
Baseline similarity 0.8550888772219306
Global similarity 0.8385569639

# Experiment 3

In [16]:
# - combined consulting 
params = {}
params["EPOCHS"] = 5
params["SAVE_PATH"] = "combined-consulting/" #"s3bert_" + params["SBERT_INIT"] + "/
params["BATCH_SIZE"] = 32
params["WARMUP_STEPS"] = 10
params["EVAL_STEPS"] = 100
params["LEARNING_RATE"] = 2e-5
params["SIMILARITY_DATASETS"] = ["consulting"]

# similarity training specific
params["USE_WEIGHTED_SIMILARITY"] = True
params["USE_MANUAL_LABELS"] = True # whether to use val-test labels that I created manually or some similarity metric
params["LOSSES"] = ["cosine"] # ["consistency", "distill", "distill2", "cosine"]
params["FEATURES"] = ['input1', 'input2']
params["USE_DISTILL"] = any('distill' in loss for loss in params["LOSSES"])
if params["USE_DISTILL"]:
    params["FEATURES"] += ['company', 'country', 'classification', 'keywords']
# FEATURES += ['similarity']
params["N"] = len(params["FEATURES"]) - 2 # number of metrics
params["FEATURE_DIM"] = 16 if params["USE_DISTILL"] else 0

# classification training specific
params["VAL_DEV_SIZE"] = 100
params["CREATE_NEW_SPLIT"] = False
params["CLASSIFICATION_DATASETS"] = ["consulting"]
params["SNIPPET_COLUMN_NAME"] = 'snippet'
params["UNFREEZE_LAYERS"] = 5
params["EXCLUDE_ENTITY_OTHER"] = True
params["OCCURENCE_CUTOFF"] = 3
params["INITIALIZED_MODEL"] = "sentence-transformers/all-MiniLM-L12-v2"
params["EVALUATION_STEPS"] = params["EVAL_STEPS"]

params["SIMILARITY_DATASET_DIR"] = '../similarity-training-data/consulting/sbert-company/'

dataset_dir = "../classification-training-data/consulting/"
model = load_model() if not params["INITIALIZED_MODEL"] else load_model(
        model=params["INITIALIZED_MODEL"])

In [17]:
column_list = ['country', 'company', 'keywords', 'classification']
column_proportions = (10, 2, 1, 1, 1)
run_pipeline(column_list, column_proportions, save_model=True)

#e=1
# Spearman
# Baseline similarity 0.9184827961965057
# Global similarity 0.9062827005287658
# MSE
# Baseline similarity 0.019708063019580228
# Global similarity 0.03229044800001069
# Setting label to weighted_similarity
# MSE
# Loading model from ./combined-consulting//
# Spearman
# Baseline similarity 0.990288613046509
# Global similarity 0.9849864986498649
# MSE
# Baseline similarity 0.006873229366766006
# Global similarity 0.0023476977664582464

# e=10
# Classification score 0.97
# Classification baseline 0.77
# Loading model from ./combined-consulting//
# Spearman
# Baseline similarity 0.9184827961965057
# Global similarity 0.8524920511365226
# MSE
# Baseline similarity 0.019708063019580228
# Global similarity 0.04196877560390803
# Setting label to weighted_similarity
# MSE
# Loading model from ./combined-consulting//
# Spearman
# Baseline similarity 0.990288613046509
# Global similarity 0.9284848484848484
# MSE
# Baseline similarity 0.006873229366766006
# Global similarity 0.009179988507821799


/Users/bartekjezierski/Desktop/master-thesis/notebooks/utils.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].loc[df[column].isnull()] = df[column].loc[df[column].isnull()


For clustering using datasets: ['consulting']
Training with multiple objectives
Reading existing dev and test triplets
e=5, embeddings from snippet, unfreezing 5 last layers, excluding Entity and Other labels, starting with sentence-transformers/all-MiniLM-L12-v2, only including words that occur at least 3 times, training data size3078

Training data size 3078
Validation data size 100
Test data size 100
Performance before fine-tuning:
For similarity using datasets: ['consulting']
['similarity']
['similarity']


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/96 [00:00<?, ?it/s]

Score at epoch 0, step -1: 1.7101563994023237


...

Score at epoch 2, step -1: 1.8215975645867477


Iteration:   0%|          | 0/96 [00:00<?, ?it/s]

Score at epoch 3, step -1: 1.8364562150508896


Iteration:   0%|          | 0/96 [00:00<?, ?it/s]

Score at epoch 4, step -1: 1.8522056038667931
Classification score 0.96
Classification baseline 0.77
Loading model from ./combined-consulting//
Spearman
Baseline similarity 0.9184827961965057
Global similarity 0.8593390127994157
MSE
Baseline similarity 0.019708063019580228
Global similarity 0.0412625300368614
Setting label to weighted_similarity
MSE
Loading model from ./combined-consulting//
Spearman
Baseline similarity 0.990288613046509
Global similarity 0.9381578157815781
MSE
Baseline similarity 0.006873229366766006
Global similarity 0.007978894998451436
